<a href="https://colab.research.google.com/github/utkarshg1/Test1/blob/main/test_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test trainig speed

In [1]:
import requests

In [2]:
def download_file(url: str):
  response = requests.get(url)
  response.raise_for_status()
  filename = url.split("/")[-1]
  with open(filename, "w") as f:
    f.write(response.text)
  return filename

In [3]:
url = "https://raw.githubusercontent.com/utkarshg1/Machine-Learning-Ravet-5-7pm/refs/heads/main/13%20Algo%20Evaluation/train_loan.csv"
print(url)

https://raw.githubusercontent.com/utkarshg1/Machine-Learning-Ravet-5-7pm/refs/heads/main/13%20Algo%20Evaluation/train_loan.csv


In [4]:
download_file(url)

'train_loan.csv'

### Load csv file

In [5]:
import pandas as pd
df = pd.read_csv("/content/train_loan.csv")
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


### Seperate X and Y

In [6]:
X = df.drop(columns = ["id", "loan_status"])
Y = df[["loan_status"]]

In [7]:
X.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3


In [8]:
Y.head()

,loan_status
0,0
1,0
2,0
3,0
4,0


In [9]:
Y.value_counts()

,count
loan_status,
0,50295
1,8350


In [10]:
cat = list(X.columns [X.dtypes == "object"])
con = list(X.columns [X.dtypes != "object"])

In [11]:
cat

['person_home_ownership',
 'loan_intent',
 'loan_grade',
 'cb_person_default_on_file']

In [12]:
con

['person_age',
 'person_income',
 'person_emp_length',
 'loan_amnt',
 'loan_int_rate',
 'loan_percent_income',
 'cb_person_cred_hist_length']

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [14]:
num_pipe = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

In [15]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")
)

In [16]:
pre = ColumnTransformer(
    [
        ("num", num_pipe, con),
        ("cat", cat_pipe, cat)
    ]
)

In [17]:
X_pre = pre.fit_transform(X)
X_pre

array([[ 1.56620019, -0.7657678 , -1.18719967, ...,  0.        ,
         0.        ,  0.        ],
       [-0.92005729, -0.21212778,  0.32804728, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.2401962 , -0.92922343,  0.8331296 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.75430679, -0.52849351,  0.58058844, ...,  0.        ,
         0.        ,  0.        ],
       [-0.92005729, -0.89758686, -0.68211735, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5716972 ,  0.28878463, -0.68211735, ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
Y_pre = Y.values.flatten()

In [19]:
Y_pre

array([0, 0, 0, ..., 1, 0, 0])

In [20]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_pre, Y_pre, test_size=0.2, random_state=42)

In [21]:
xtrain.shape

(46916, 22)

In [22]:
xtest.shape

(11729, 22)

In [23]:
from sklearn.model_selection import cross_val_score
import time

In [24]:
def evaluate_model(model, xtrain, ytrain, xtest, ytest, n_jobs=-1):
  start = time.perf_counter()
  model.fit(xtrain, ytrain)
  train_score = model.score(xtrain, ytrain)
  test_score = model.score(xtest, ytest)
  scores = cross_val_score(model, xtrain, ytrain, cv=5, scoring="f1_macro", n_jobs=n_jobs)
  stop = time.perf_counter()
  elapsed = round(stop - start, 4)
  name = type(model).__name__
  return {
      "name": name,
      "model": model,
      "train_score": train_score,
      "test_score": test_score,
      "cv_score": scores.mean(),
      "model_time": elapsed
  }


In [25]:
def evaluate_multiple(models: list, xtrain, ytrain, xtest, ytest):
  start = time.perf_counter()
  res = []
  for model in models:
    r = evaluate_model(model, xtrain, ytrain, xtest, ytest)
    res.append(r)
    print(r)
  res_df = pd.DataFrame(res)
  res_sorted = res_df.sort_values(by="cv_score", ascending=False).reset_index(drop=True)
  best_model = res_sorted.loc[0, "model"]
  stop = time.perf_counter()
  elapsed = round(stop - start, 4)
  print(f"Total time: {elapsed}")
  return best_model, res_sorted

In [26]:
%pip install xgboost

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier

In [28]:
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    HistGradientBoostingClassifier(),
    XGBClassifier()
]

In [29]:
best_model, res = evaluate_multiple(models, xtrain, ytrain, xtest, ytest)

{'name': 'LogisticRegression', 'model': LogisticRegression(), 'train_score': 0.9121621621621622, 'test_score': 0.9114161480092079, 'cv_score': np.float64(0.792751366871291), 'model_time': 8.4214}
{'name': 'DecisionTreeClassifier', 'model': DecisionTreeClassifier(), 'train_score': 1.0, 'test_score': 0.9092846790007674, 'cv_score': np.float64(0.8261974818802587), 'model_time': 3.5476}
{'name': 'HistGradientBoostingClassifier', 'model': HistGradientBoostingClassifier(), 'train_score': 0.9554522977235911, 'test_score': 0.9517435416489044, 'cv_score': np.float64(0.8898638156088735), 'model_time': 10.4195}
{'name': 'XGBClassifier', 'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
             

In [30]:
res

,name,model,train_score,test_score,cv_score,model_time
0,HistGradientBoostingClassifier,HistGradientBoostingClassifier(),0.955452,0.951744,0.889864,10.4195
1,XGBClassifier,"XGBClassifier(base_score=None, booster=None, c...",0.966174,0.951999,0.889798,3.3433
2,DecisionTreeClassifier,DecisionTreeClassifier(),1.000000,0.909285,0.826197,3.5476
3,LogisticRegression,LogisticRegression(),0.912162,0.911416,0.792751,8.4214


In [31]:
best_model

HistGradientBoostingClassifier()

In [32]:
from joblib import Parallel, delayed

In [33]:
def evaluate_parallel(models: list, xtrain, ytrain, xtest, ytest):
  start = time.perf_counter()
  results = Parallel(n_jobs=-1)(
      delayed(evaluate_model)(model, xtrain, ytrain, xtest, ytest, n_jobs=1)
      for model in models
  )
  res_df = pd.DataFrame(results)
  res_sorted = res_df.sort_values(by="cv_score", ascending=False).reset_index(drop=True)
  best_model = res_sorted.loc[0, "model"]
  stop = time.perf_counter()
  elapsedd = round(stop - start, 4)
  print(f"Total time: {elapsedd}")
  return best_model, res_sorted

In [34]:
best_model, res = evaluate_parallel(models, xtrain, ytrain, xtest, ytest)

Total time: 28.7756


In [35]:
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [36]:
res

,name,model,train_score,test_score,cv_score,model_time
0,XGBClassifier,"XGBClassifier(base_score=None, booster=None, c...",0.966174,0.951999,0.889798,14.9472
1,HistGradientBoostingClassifier,HistGradientBoostingClassifier(),0.955729,0.952340,0.889729,25.1662
2,DecisionTreeClassifier,DecisionTreeClassifier(),1.000000,0.910564,0.824595,4.3122
3,LogisticRegression,LogisticRegression(),0.912162,0.911416,0.792751,2.5426
